1b Fundamentals of Data Structures and Algorithms

Assignment 5

1. Graph Analysis

Write a script that can accept an adjacency matrix as input, and construct a node-based representation of a graph.  

Next, write a method that takes two nodes as arguments, n1 and n2, and returns the number of nodes that are equidistant from n1 and n2.  These are the nodes for which the shortest paths to n1 and n2 have the same length.  

Hint: run two breadth-first searches, one from n1 and one from n2.

In [233]:
class GraphNode(object):
    def __init__(self, value = None):
        self.value = value
        self.connections = []
        
    def add_connections(self, other):
        if other not in self.connections:
            self.connections.append(other)
        if self not in other.connections:
            other.connections.append(self)
            
    def get_connections(self):
        return self.connections  

def node_based(adj_matrix): 
    # the adjacent matrix is a square by definition, therefore row size = column size
    matrix_nodes = adj_matrix.shape[0]
    row_size = adj_matrix.shape[0]
    column_size = adj_matrix.shape[1]
    
    # turn each element in adj_matrix to graph node if node exists and append node to new list: "graph"
    graph = []
    for x in range(0, matrix_nodes):
        x = GraphNode()
        graph.append(x)

    row = 0
    column = 0
    while row < row_size:
        while column < column_size:
            if adj_matrix.iloc[row, column] == 1:
                graph[row].add_connections(graph[column]) # connect graph nodes
                column += 1
            else:
                column += 1
        row += 1
        column = 0          
    return graph

# creates list of next-generation nodes
def generation(set_of_connections, neglect_list): # neglect_list is input of nodes already visited
    nexgen = []
    for child in set_of_connections:
        nexgen += child.get_connections() # create list of all next-generation nodes
    nexgen_as_set = set(nexgen)
    neglect_list = set(neglect_list)
    intersection = nexgen_as_set.difference(neglect_list) # find nodes that are in common between two lists
    final_list = list(intersection) 
    final_list = [x for x in nexgen if x not in neglect_list]
    return final_list # returns list of subsequent generation without parent nodes

def compare_generations(gen_1, gen_2):
    gen1_as_set = set(gen_1)
    gen2_as_set = set(gen_2)
    intersection = gen1_as_set.intersection(gen2_as_set) # find nodes that are in common between two lists
    equi_nodes = list(intersection) 
    return equi_nodes
    
def breadth_traverse(n1, n2): # assuming n1 and n2 are already graph nodes
    equi_nodes_final = []
    gen_1 = n1.get_connections()
    gen_2 = n2.get_connections()
    
    neglect_list = [n1, n2]
    num_nodes = 1
    counter = 0
    while num_nodes > 0:
        nodes = compare_generations(gen_1, gen_2) 
        num_nodes = len(nodes)
        equi_nodes_final.append(num_nodes)
        neglect_list += (gen_1 + gen_2)
        gen_1 = generation(gen_1, neglect_list) # reassign variable to continue loop until either list is empty
        gen_2 = generation(gen_2, neglect_list)
        counter += 1
    return sum(equi_nodes_final)
    

In [234]:
import pandas as pd

adj_matrix = pd.read_csv('adj.txt', sep = ' ', header = None) # import adj_matrix text file

adj_matrix
adj_matrix = adj_matrix.drop(columns = [25, 26]) # clean adj_matrix to remove NANs and excessive columns

In [235]:
final_graph = node_based(adj_matrix)

assert(breadth_traverse(final_graph[0], final_graph[1]) == 11)
assert(breadth_traverse(final_graph[0], final_graph[5]) == 12)
assert(breadth_traverse(final_graph[1], final_graph[8]) == 13)